# Scripps Automated Shore Station (SASS) Self-Calibrating SeapHOx (SCS) live data viewer
### This script scrubs .dat files from the SCCOOS dr: https://sccoos.org/dr/data/. The SCS measures pH, oxygen, salinity and temperature. The pH values plotted here are calibrated using an average of the coefficients established pre-deployment and from the first automated tris calibration on 18 Feb 2022. Before using and sharing these values, please get permission from the author. A new tab delimited text file will be produced every time the script runs, with the current timestamp of the time the file was created. All time zones associated with this script are in UTC.
### Created by: Taylor Wirth twirth@ucsd.edu
### Last edit: 3 March 2022

In [1]:
# load the .py scripts
import pandas as pd
import os
import glob
%load_ext autoreload
%autoreload 1
%aimport get_recent_dr
%aimport get_all_dr
%aimport pHext_from_Vext_k0ext
%aimport pHint_from_Vint_k0int
%aimport plot_all

The last hour of data and plot of all the data will show below. Please wait!

In [2]:
# Get all the files for the Scripps Pier SCS. get_all_dr.py looks and grabs all data files for the Scripps Pier SCS, 
# then converts it into a pandas DataFrame. 
# Printed below is the last hour of data. This may take a few seconds depending on last data download!
path = 'https://sccoos.org/dr/data/scripps_pier_scs/'

# if no txt file of downloaded data, get all of the data:
txt = glob.glob('*.txt')
if txt:
    df, col_names = get_recent_dr.get_recent_dr(path)
    # delete current txt file
    os.remove(txt[0])
else:
    df, col_names = get_all_dr.get_all_dr(path)

# create new tab delimited file to download
from datetime import datetime
now = datetime.utcnow()
current_time = now.strftime('%Y%m%d%H%M%S')
df.to_csv('SIOpierSCS_download_'+current_time+'.txt', header=col_names, index=None, sep='\t')

df.tail(6)

,internet_datetime,IP,samp_type,samp_num,calib_num,calib_rep,date,time,vbatt,vtherm,...,Bpot,Ramp,Raw_Temp,SBEtemp,SBEcond,SBEsal,SBEday,SBEmon,SBEyear,SBEtime
2927,2022-03-14T00:50:27Z,172.16.117.226,0,3350,1,0,2022/03/14,00:50:10,15.59,0.54380,...,999.7,650.8,322.0,16.3313,4.25567,33.5196,13,Mar,2022,17:04:15
2928,2022-03-14T01:00:27Z,172.16.117.226,0,3351,1,0,2022/03/14,01:00:10,15.59,0.54354,...,1000.6,650.7,321.8,16.3457,4.25723,33.5212,13,Mar,2022,17:14:15
2929,2022-03-14T01:10:27Z,172.16.117.226,0,3352,1,0,2022/03/14,01:10:10,15.59,0.54405,...,1001.7,650.6,322.7,16.2982,4.25237,33.5184,13,Mar,2022,17:24:15
2930,2022-03-14T01:20:27Z,172.16.117.226,0,3353,1,0,2022/03/14,01:20:10,15.59,0.54425,...,1002.4,650.7,323.4,16.2670,4.24926,33.5173,13,Mar,2022,17:34:15
2931,2022-03-14T01:30:27Z,172.16.117.226,0,3354,1,0,2022/03/14,01:30:10,15.59,0.54490,...,1002.3,650.7,323.5,16.2798,4.25027,33.5154,13,Mar,2022,17:44:15
2932,2022-03-14T01:40:27Z,172.16.117.226,0,3355,1,0,2022/03/14,01:40:10,15.59,0.54476,...,1002.7,650.8,323.4,16.2862,4.25090,33.5156,13,Mar,2022,17:54:15


In [3]:
import numpy as np
caldf = df.loc[df['samp_type']==1]
df = df.drop(df[df.samp_type==1].index) # remove calibration samples from the data frame

In [4]:
print('\n\033[1m'+'Calibration coefficients:'+'\033[0m')
k0int = -0.3961671
k0ext = -1.3843647
print('k0int from predeployment =', k0int)
print('k0ext from predeployment =', k0ext)

# calibration coefficients from first automated tris calibration on Feb 18, 2022
k0int_tris1 = -0.3970633
k0ext_tris1 = -1.3848532
print('k0int from tris 18 Feb 2022 =', k0int_tris1)
print('k0ext from tris 18 Feb 2022=', k0ext_tris1)

# calibration coefficients from first automated tris calibration on Mar 4, 2022
k0int_tris2 = -0.3965293
k0ext_tris2 = -1.3848580
print('k0int from tris 4 Mar 2022 =', k0int_tris2)
print('k0ext from tris 4 Mar 2022=', k0ext_tris2)

k0int_tris = (k0int_tris1+k0int_tris2)/2
k0ext_tris = (k0ext_tris1+k0ext_tris2)/2

k0intmean = (k0int+k0int_tris1+k0int_tris2)/3
k0extmean = (k0ext+k0ext_tris1+k0ext_tris2)/3
print('mean k0int = {0:.7f}'.format(k0intmean))
print('mean k0ext = {0:.7f}'.format(k0extmean))

datetime = list(df['date'] + ' ' + df['time']) # combine date and time to use for plots
Vint = df.vint.astype(float)
Vext = df.vext.astype(float)
T_C = df.SBEtemp.astype(float)
sal = df.SBEsal.astype(float)
pHint_SBE = pHint_from_Vint_k0int.pHint_from_Vint_k0int(k0intmean, Vint, T_C)
pHint_tris = pHint_from_Vint_k0int.pHint_from_Vint_k0int(k0int_tris,Vint, T_C)
pHext_SBE = pHext_from_Vext_k0ext.pHext_from_Vext_k0ext(k0extmean, Vext, T_C, sal)
pHext_tris = pHext_from_Vext_k0ext.pHext_from_Vext_k0ext(k0ext_tris, Vext, T_C, sal)


Calibration coefficients:
k0int from predeployment = -0.3961671
k0ext from predeployment = -1.3843647
k0int from tris 18 Feb 2022 = -0.3970633
k0ext from tris 18 Feb 2022= -1.3848532
k0int from tris 4 Mar 2022 = -0.3965293
k0ext from tris 4 Mar 2022= -1.384858
mean k0int = -0.3965866
mean k0ext = -1.3846920


In [5]:
plot_all.plot_all(datetime,pHint_SBE,pHext_SBE,df.O2con,df.O2sat,df.SBEtemp,df.O2temp,df.pHtemp,df.SBEsal,df.press)